In [1]:
import faker
import pyodbc
import datetime
import gc
import random

SQL_DRIVER = r"DRIVER={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={path};"
DB_FILE="C:/Users/Student/Documents/DBMS FINAL PROJECT.accdb"

In [52]:
conn = SQL_DRIVER.format(path = DB_FILE)
db = pyodbc.connect(conn, autocommit = False)

test_query = """
    SELECT TOP 5 * FROM order_details;
"""

cursor = db.cursor()
result = cursor.execute(test_query).fetchall()
print(f"result: {result}")

result: [(1, 1, 109), (2, 2, 108), (3, 2, 124), (4, 2, 117), (5, 2, 129)]


In [53]:
get_menu_items_query = """
    SELECT menu_item_id, item_name, price FROM menu_items;
"""

menu_items = {item[0] : (item[1], item[2]) for item in cursor.execute(get_menu_items_query).fetchall()}
print(menu_items)

{101: ('Hamburger', Decimal('12.9500')), 102: ('Cheeseburger', Decimal('13.9500')), 103: ('Hot Dog', Decimal('9.0000')), 104: ('Veggie Burger', Decimal('10.5000')), 105: ('Mac & Cheese', Decimal('7.0000')), 106: ('French Fries', Decimal('7.0000')), 107: ('Orange Chicken', Decimal('16.5000')), 108: ('Tofu Pad Thai', Decimal('14.5000')), 109: ('Korean Beef Bowl', Decimal('17.9500')), 110: ('Pork Ramen', Decimal('17.9500')), 111: ('California Roll', Decimal('11.9500')), 112: ('Salmon Roll', Decimal('14.9500')), 113: ('Edamame', Decimal('5.0000')), 114: ('Potstickers', Decimal('9.0000')), 115: ('Chicken Tacos', Decimal('11.9500')), 116: ('Steak Tacos', Decimal('13.9500')), 117: ('Chicken Burrito', Decimal('12.9500')), 118: ('Steak Burrito', Decimal('14.9500')), 119: ('Chicken Torta', Decimal('11.9500')), 120: ('Steak Torta', Decimal('13.9500')), 121: ('Cheese Quesadillas', Decimal('10.5000')), 122: ('Chips & Salsa', Decimal('7.0000')), 123: ('Chips & Guacamole', Decimal('9.0000')), 124: ('

In [54]:
get_last_order_details_id_query = """
    SELECT TOP 1 order_details_id, order_id FROM order_details ORDER BY order_details_id DESC;
"""

last_order_details_id, last_order_id = cursor.execute(get_last_order_details_id_query).fetchone()
print(f"last order details id: {last_order_details_id}")
print(f"last order id: {last_order_id}")

last order details id: 12234
last order id: 5370


In [59]:
print(cursor.execute(""" SELECT order_date, order_time FROM orders WHERE order_id = 40""").fetchone())

(datetime.datetime(2023, 1, 1, 0, 0), datetime.datetime(1899, 12, 30, 17, 14, 36))


In [ ]:
fkr = faker.Faker()
menu_item_ids = set(key for key in menu_items.keys())

orders = []
for _ in range(100_000):  # Generate 100000 fake orders
    order_date = fkr.date_time_between_dates(datetime.datetime(2023, 4, 1), datetime.datetime.now())
    order_time = datetime.datetime.strptime(fkr.time(), "%H:%M:%S").strftime("%H:%M:%S %p")
    orders.append((order_date, order_time))

orders.sort()
print(orders[0])

(datetime.datetime(2023, 4, 1, 0, 4, 28), '14:26:44 PM')


In [ ]:
# Insert 100000 rows into the table
for order in orders:
    cursor.execute("INSERT INTO orders (order_date, order_time) VALUES (?, ?)", (order[0], order[1]))

cursor.execute("SELECT TOP 100 * FROM orders ORDER BY order_id DESC")  # Adjust the ORDER BY clause as needed
inserted_rows = cursor.fetchall()

    # Display the inserted rows
print("Inserted rows before commit:")
for row in inserted_rows:
    print(row)


In [90]:
current_order_id = last_order_id + 1
order_details = []
current_order_details_id = last_order_details_id + 1
for order in orders:
    for _ in range(random.randint(1, 10)):
        menu_item_id = fkr.random_int(min=101, max=132)
        order_details.append((current_order_details_id, current_order_id, menu_item_id))
        current_order_details_id += 1
    current_order_id += 1


In [ ]:
print(f"order details count: {len(order_details)}")
for i in range(1000):
    print(order_details[i])

In [ ]:
for order_detail in order_details:
    cursor.execute("INSERT INTO order_details (order_details_id, order_id, menu_item_id) VALUES (?, ?, ?)", order_detail)

Inserted order rows before commit:
(105370, datetime.datetime(2024, 11, 26, 18, 57, 23), datetime.datetime(1899, 12, 30, 7, 37, 13))
(105369, datetime.datetime(2024, 11, 26, 18, 44, 48), datetime.datetime(1899, 12, 30, 20, 36, 7))
(105368, datetime.datetime(2024, 11, 26, 18, 38, 34), datetime.datetime(1899, 12, 30, 14, 55, 38))
(105367, datetime.datetime(2024, 11, 26, 18, 36, 28), datetime.datetime(1899, 12, 30, 2, 20, 44))
(105366, datetime.datetime(2024, 11, 26, 18, 32, 29), datetime.datetime(1899, 12, 30, 4, 38, 29))
(105365, datetime.datetime(2024, 11, 26, 18, 31, 28), datetime.datetime(1899, 12, 30, 0, 32, 50))
(105364, datetime.datetime(2024, 11, 26, 18, 30, 45), datetime.datetime(1899, 12, 30, 8, 22, 20))
(105363, datetime.datetime(2024, 11, 26, 18, 21, 1), datetime.datetime(1899, 12, 30, 15, 36, 53))
(105362, datetime.datetime(2024, 11, 26, 18, 1, 19), datetime.datetime(1899, 12, 30, 18, 35, 25))
(105361, datetime.datetime(2024, 11, 26, 17, 58, 32), datetime.datetime(1899, 12, 

In [94]:
cursor.execute("SELECT TOP 10 * FROM orders ORDER BY order_id DESC")  # Adjust the ORDER BY clause as needed
inserted_rows = cursor.fetchall()

    # Display the inserted rows
print("Inserted order rows before commit:")
for row in inserted_rows:
    print(row)

cursor.execute("SELECT TOP 10 * FROM order_details ORDER BY order_details_id DESC")  # Adjust the ORDER BY clause as needed
inserted_rows = cursor.fetchall()

    # Display the inserted rows
print("Inserted order detail rows before commit:")
for row in inserted_rows:
    print(row)

Inserted order rows before commit:
(105370, datetime.datetime(2024, 11, 26, 18, 57, 23), datetime.datetime(1899, 12, 30, 7, 37, 13))
(105369, datetime.datetime(2024, 11, 26, 18, 44, 48), datetime.datetime(1899, 12, 30, 20, 36, 7))
(105368, datetime.datetime(2024, 11, 26, 18, 38, 34), datetime.datetime(1899, 12, 30, 14, 55, 38))
(105367, datetime.datetime(2024, 11, 26, 18, 36, 28), datetime.datetime(1899, 12, 30, 2, 20, 44))
(105366, datetime.datetime(2024, 11, 26, 18, 32, 29), datetime.datetime(1899, 12, 30, 4, 38, 29))
(105365, datetime.datetime(2024, 11, 26, 18, 31, 28), datetime.datetime(1899, 12, 30, 0, 32, 50))
(105364, datetime.datetime(2024, 11, 26, 18, 30, 45), datetime.datetime(1899, 12, 30, 8, 22, 20))
(105363, datetime.datetime(2024, 11, 26, 18, 21, 1), datetime.datetime(1899, 12, 30, 15, 36, 53))
(105362, datetime.datetime(2024, 11, 26, 18, 1, 19), datetime.datetime(1899, 12, 30, 18, 35, 25))
(105361, datetime.datetime(2024, 11, 26, 17, 58, 32), datetime.datetime(1899, 12, 

In [95]:
# Commit the transaction
db.commit()
print("Transaction committed successfully.")

Transaction committed successfully.


In [96]:
cursor.close()
db.close()